In [1]:
!python -m pip install -q transformers
!python -m pip install -q pillow
!python -m pip install -q loguru
!python -m pip install -q pydantic
!python -m pip install -q pandas
!python -m pip install -q opencv-python


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from PIL import Image
import torch
from transformers import OwlViTProcessor, OwlViTForObjectDetection

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
processor = OwlViTProcessor.from_pretrained("google/owlvit-large-patch14")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-large-patch14").to(device)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
model.eval()

OwlViTForObjectDetection(
  (owlvit): OwlViTModel(
    (text_model): OwlViTTextTransformer(
      (embeddings): OwlViTTextEmbeddings(
        (token_embedding): Embedding(49408, 768)
        (position_embedding): Embedding(16, 768)
      )
      (encoder): OwlViTEncoder(
        (layers): ModuleList(
          (0-11): 12 x OwlViTEncoderLayer(
            (self_attn): OwlViTAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): OwlViTMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_

In [5]:
model.device

device(type='cuda', index=0)

In [6]:

from statistics import mean
import time
import torch
from utils.protocols import OwlVit
from utils.utils import convert_model_detection
from utils.video import read_vid_batch, read_video
from utils.utils import get_file_name, get_gpu_name
from loguru import logger
from datetime import datetime

BATCH_SIZE = 15

MODEL= f"owlvit-large-p14-hf-batch-{BATCH_SIZE}"

text = "face"
BASE_DIR = "experiments/owlvit"


def process_video(video):
    start_time = datetime.now()
    target_sizes = None
    results = {}
    pre_processing_times = []
    inference_times = []
    post_processing_times = []
    last = None
    for batch in read_vid_batch(video, batch_size=BATCH_SIZE):
        start = time.time()
        frame_ids = list(batch.keys())
        frames =  [Image.fromarray(frame.astype("uint8")) for frame in batch.values()]
        prompts = [text for _ in frames]
        target_sizes = [frame.size for frame in frames]
        inputs = processor(images=frames, text=prompts, return_tensors="pt").to(device)
        
        pre_processing_times.append(time.time()-start)
        
        start = time.time()
        #with torch.no_grad():
        with torch.inference_mode():
            outputs = model(**inputs)
        inference_times.append(time.time()-start)
        
        start = time.time()
        detections = processor.post_process_object_detection(outputs=outputs, threshold=0.1, target_sizes=target_sizes)
        batch_result =  {frame_id: convert_model_detection(detection) for frame_id, detection in zip(frame_ids, detections)}
        results = {**results, **batch_result}
        post_processing_times.append(time.time()-start)
        torch.cuda.empty_cache()
        last = frame_ids[-1]

    end_time = datetime.now()
    n_frames = len(results)
    
    exp =  OwlVit(
        model=MODEL,
        gpu=get_gpu_name(),
        video_file=video,
        frames=None,
        batch_size=BATCH_SIZE,
        n_frames=n_frames,
        
        pre_processing_time=sum(pre_processing_times),
        inference_time=sum(inference_times),
        post_processing_time=sum(post_processing_times),
        video_processing_time=(end_time-start_time).seconds,
       
        start_time=start_time.isoformat(),
        end_time=end_time.isoformat(),
        record_file=get_file_name(BASE_DIR, start, MODEL, video),
        
        data=results,
        prompt=text,
        
    )
    exp.save()
    exp.log()
    torch.cuda.empty_cache()
    return exp

In [ ]:
results = []
for i in range(1):
    vid_hd = process_video("data/720.mp4")
    results.append(vid_hd)
    vid_fhd = process_video("data/1080.mp4")
    results.append(vid_fhd)

2024-06-12 11:31:00.405 | INFO     | utils.protocols:log:113 - data/720.mp4 | frames=283 | model_fps=3.668199261126982 | inference_time=77.14957118034363 | preprocess_time=18.242178440093994


In [ ]:
columns = results[0].columns
rows = [result.row for result in results]

In [ ]:
import pandas as pd

df = pd.DataFrame(rows, columns=columns)
df.head(10)

In [ ]:
from datetime import datetime
now = datetime.now()
csv_file = f"zz-{MODEL}-{get_gpu_name()}-{now.day}-{now.hour}-{now.minute}.csv"
df.to_csv(csv_file)